# 구글 드라이브 연동

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 원 핫 인코딩 적용

In [3]:
import pandas as pd

DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/산대특/ch10/data/'
class2 = pd.read_csv(DATA_PATH + "class2.csv")
class2

,Unnamed: 0,id,tissue,class,class2,x,y,r
0,0,CID000,C,CIRC,N,535.0,475.0,192.0
1,1,CID001,A,CIRA,N,433.0,268.0,58.0
2,2,CID002,A,CIRA,I,NaN,NaN,NaN
3,3,CID003,C,CIRC,B,NaN,NaN,NaN
4,4,CID004,F,CIRF,I,488.0,145.0,29.0
5,5,CID005,C,CIRC,B,532.0,199.0,21.0


In [4]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder() 

# pd.get_dummies()

# train_x = label_encoder.fit_transform(class2['class'])
# train_x = 

array([1, 0, 0, 1, 2, 1])

In [6]:
train_x = pd.get_dummies(class2)
train_x

,Unnamed: 0,x,y,r,id_CID000,id_CID001,id_CID002,id_CID003,id_CID004,id_CID005,tissue_A,tissue_C,tissue_F,class_CIRA,class_CIRC,class_CIRF,class2_B,class2_I,class2_N
0,0,535.0,475.0,192.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1
1,1,433.0,268.0,58.0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1
2,2,NaN,NaN,NaN,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0
3,3,NaN,NaN,NaN,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0
4,4,488.0,145.0,29.0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0
5,5,532.0,199.0,21.0,0,0,0,0,0,1,0,1,0,0,1,0,1,0,0


- 자연어 처리를 할 때는 원-핫 인코딩 사용X
- 직교를 이룬다 --> 서로 독립적인 관계가 된다.
- 대안을 찾자 : 워드투벡터, 글로브, 패스트텍스트 등을 사용한다. 

## 횟수 기반 임베딩
- CountVectorizer, 교재(p400)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is last chance.',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance',
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

#카운트벡터는 쓰지 않는다.

{'this': 13,
 'is': 7,
 'last': 8,
 'chance': 2,
 'and': 0,
 'if': 6,
 'you': 15,
 'do': 3,
 'not': 10,
 'have': 5,
 'will': 14,
 'never': 9,
 'get': 4,
 'any': 1,
 'one': 11,
 'please': 12}

In [9]:
vect = CountVectorizer(stop_words=["and", "is", "please", "this"]).fit(corpus)
vect.vocabulary_

{'last': 6,
 'chance': 1,
 'if': 5,
 'you': 11,
 'do': 2,
 'not': 8,
 'have': 4,
 'will': 10,
 'never': 7,
 'get': 3,
 'any': 0,
 'one': 9}

- TF-IDF, 교재(p400)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
doc = ['I like machine learning', 'I love deep learning', 'I run everyday']
tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = (tfidf_matrix * tfidf_matrix.T)
print ('유사도를 위한', str(doc_distance.get_shape()[0]), 'x', str(doc_distance.get_shape()[1]), '행렬을 만들었습니다.')
print(doc_distance.toarray())

유사도를 위한 3 x 3 행렬을 만들었습니다.
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


### 예측 기반 임베딩 
- 워드 투 벡터
  + 신경망 알고리즘의 하나임
- and I love this one
- one을 입력 --> 출력층 나머지 4개 단어 구성
- 각 단어 간의 코사인 유사도를 통해 해당 단어의 의미를 수치화한다.

In [12]:
%cd /content/drive/MyDrive/Colab Notebooks/산대특/ch10/data

/content/drive/MyDrive/Colab Notebooks/산대특/ch10/data


In [13]:
!ls

 class2.csv	     glove.6B.300d.txt	'IMDB Dataset.zip'   test.csv
 example.txt	     glove.6B.50d.txt	 peter.txt	     train.csv
 glove.6B.100d.txt   glove.6B.zip	 spam.csv	     wiki.ko.vec
 glove.6B.200d.txt  'IMDB Dataset.csv'	 spa.txt


In [17]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action='ignore')
import nltk
nltk.download('punkt')

import gensim
from gensim.models import Word2Vec

sample = open("peter.txt", "r", encoding="UTF-8")
s = sample.read()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [19]:
#줄바꿈을 공백으로 변환
f = s.replace("\n", " ")
data = []
for i in sent_tokenize(f):
  temp = []
  for j in word_tokenize(i):
    temp.append(j.lower())
    data.append(temp)

data

[['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  't

### 단어간 유사성을 확인

In [25]:
#sg = 0, CBOW 알고리즘 선택
#sg = 1, skip-gram 알고리즘을 선택한다
model1 = gensim.models.Word2Vec(data,
                               min_count=1,
                               size=100,
                               window=5,
                               sg=0)

print("Cosine similarity between 'peter' " +
                 "'wendy' - CBOW : ", 
      model1.similarity('peter', 'hook')) 

Cosine similarity between 'peter' 'wendy' - CBOW :  0.9989162


In [26]:
#sg = 0, CBOW 알고리즘 선택
#sg = 1, skip-gram 알고리즘을 선택한다
model2 = gensim.models.Word2Vec(data,
                               min_count=1,
                               size=100,
                               window=5,
                               sg=1)

print("Cosine similarity between 'peter' " +
                 "'wendy' - Skip Gram : ", 
      model1.similarity('peter', 'hook')) 

Cosine similarity between 'peter' 'wendy' - Skip Gram :  0.9989162


### 패스트텍스트
- 워드 투 벡터 단점 : 사전에 모르는 단어는 학습이 어려움
- 워드 투 벡터의 단점을 보완함.
- 소파 위에 있는 고양이가 낮잠을 잔다.
  + 워드 투 벡터 ==> 문장 1개 사용
  + N-그램 --> 세트를 3개 정도 

In [30]:
from gensim.test.utils import common_texts 
from gensim.models import FastText 

corpus_fname = 'peter.txt'
corpus = [sent.strip().split(" ") for sent in open(corpus_fname, 'r', encoding='utf-8').readlines()]
model = FastText(corpus, size=4, window=3, min_count = 1, iter = 10)

sim_score = model.wv.similarity('peter', 'wendy')
print(sim_score)

-0.5548141


In [32]:
sim_score = model.wv.similarity('peter', 'hook')
print(sim_score)

-0.1171107


## 한글 모델 사용
- 사전 훈련된 모델을 호출하자.


In [37]:
from __future__ import print_function
from gensim.models import KeyedVectors

model_kr = KeyedVectors.load_word2vec_format('wiki.ko.vec')
print(model_kr)

In [49]:
find_similar_to = '대한민국'

for similar_word in model_kr.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]
    ))

Word: 주대한민국, Similarity: 0.85
Word: 한민국, Similarity: 0.81
Word: 대한민국와, Similarity: 0.77
Word: 대한민국건국, Similarity: 0.74
Word: 대한민국육군, Similarity: 0.73
Word: 대한민국민, Similarity: 0.73
Word: 대한민국적, Similarity: 0.72
Word: 대한민국인, Similarity: 0.71
Word: 대한민국토박이, Similarity: 0.71
Word: 대한민국재향군인회, Similarity: 0.71


In [51]:
similarities = model_kr.wv.most_similar(positive=['일본', '중국'], negative=['한국'])
print(similarities)

[('중화인민공하국', 0.6181370615959167), ('중화인민공화국', 0.5936577320098877), ('톈랴오', 0.586189329624176), ('톈중', 0.5725148916244507), ('대만', 0.5692499876022339), ('톈진의', 0.569195032119751), ('훙후', 0.5680252313613892), ('중국등', 0.5665101408958435), ('광저우시', 0.5654244422912598), ('랴오중카이', 0.5621193647384644)]


### 횟수/예측 기반 임베딩
- 횟수 기반 임베딩의 단점
- 예측 기반 임베딩의 단점
- Glove : 위 두개의 알고리즘 단점을 보완.

In [53]:
!ls

 class2.csv	     glove.6B.300d.txt	'IMDB Dataset.zip'   test.csv
 example.txt	     glove.6B.50d.txt	 peter.txt	     train.csv
 glove.6B.100d.txt   glove.6B.zip	 spam.csv	     wiki.ko.vec
 glove.6B.200d.txt  'IMDB Dataset.csv'	 spa.txt


In [55]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = 'glove.6B.100d.txt'
word2vec_glove_file = get_tmpfile('glove6B.100d.word2vec.txt')
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [56]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)
model.most_similar('bill')

[('legislation', 0.8072140216827393),
 ('proposal', 0.7306863069534302),
 ('senate', 0.7142540812492371),
 ('bills', 0.7044401168823242),
 ('measure', 0.6958035230636597),
 ('passed', 0.6906244158744812),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663139462471008)]

## p419 트랜스포머 어텐션
- 자연어 알고리즘의 가장 중요한 핵심
- 지금 현재 
  + 트랜스포머 알고리즘 시대
- 단어 번역
  + 새로운 단어가 들어오면, 다른 모든 단어와 비교해서 번역
  + 병렬구조로 처리 --> 속도도 향상 / 정확도 향상
- 영향력 
  + GPT-123 계열
  + Bert 계열

- 교재 p422 seq2seq 먼저 읽고 트랜스포머 어텐션 읽을 것 
  + 파라미터
  + d-model 입력과 출력의 크기 사전 지정:차원
  + num_layers :층을 다층구조로 쌓는다.
  + num_heads : 여러개로 분할해서 병렬로 어텐션 수행
  + dff : 신경망의 크기
- 트렌스포머 입력하는 방식 중 가장 대표적인 것이
  + 포지셔널 인코딩
  + 기존 모델은 순차적으로 처리하는 것이 핵심
  + 트랜스포머, 각 단어의 위치 정보를
- 파이썬 Dictionary의 Key-Value 값을 생각하자

+ 자연어 공부하려고 할 때 필수적으로 공부하면 좋은 페이지
 - https://wikidocs.net/book/2155